In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from seaborn import histplot, pointplot, lineplot, barplot

In [ ]:
filename = '/Users/vdk/MST_work/data_muons/mono_proton_simtel/proton_20deg_0deg_run3___cta-prod6-paranal-2147m-Paranal-nshow-3000-0-LSTs-1-MSTs-0-SSTs-NSCAT_5_CSCAT_300_EMIN_1_EMAX_200_after_muontag.simtel.zst'
#filename = '/Users/vdk/MST_work/data_muons/mono_muon_simtel/muon_20deg_0deg_run0___cta-prod6-paranal-2147m-Paranal-0-LSTs-1-MSTs-0-SSTs-NSCAT_1_CSCAT_8_EMIN_0.005_EMAX_1_FIXCHI_600_VIEWCONE_3_after_muontag.simtel.zst'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)



for i,j in enumerate(event_iterator):
    if i < 200:
        event = j
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(
                image_extractor_type="NeighborPeakWindowSum",
                subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
        fig.subplots_adjust(hspace=0.2)
        ax = axes[0]
        camgeom = source.subarray.tel[1].camera.geometry
        title="TrImg run {} event {} fit {} width {}".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = ax)
        disp.image = j.simulation.tel[1].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        bx = axes[1]
        for pix_id in range(0,1764):
            bx.plot(j.r0.tel[1].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
        bx.set_ylabel("ADC counts")
        bx.set_xlabel("Samples")
        bx.set_title("Waveforms")
        #plt.show()
        plt.savefig(f"/Users/vdk/MST_ProtonImages_NewQualityCheck/TrueImage+Waveform_mstPROTON_event{j.index.event_id}_obs{j.index.obs_id}_numer{i}")
        #plt.savefig(f"/Users/vdk/TrueImage+Waveform_mstMUON_event{j.index.event_id}_obs{j.index.obs_id}")
        #break
    if i == 150:
        break

plt.close()

In [ ]:
# Sampling rate for MST
source.subarray.tel[1].camera.readout.sampling_rate.to_value(1/u.s)
source.subarray.tel[1].camera.geometry.neighbor_matrix_sparse
min(event.dl1.tel[1].peak_time)

In [ ]:
plt.pcolormesh(event.r0.tel[1].waveform[0])  # note the [0] is for channel 0
plt.colorbar()
plt.xlabel("sample number")
plt.ylabel("Pixel_id")

In [ ]:
for pix_id in range(185,195):
    plt.plot(event.r0.tel[1].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
#plt.legend()

In [ ]:
test_image = np.zeros((1764,25))
test_true_image = np.zeros(1764)
edit_image = event.simulation.tel[1].true_image.copy()
edit_image.shape

In [ ]:
plt.figure(figsize = (10,7))
camgeom = source.subarray.tel[1].camera.geometry
#title="CT{}, run {} event {}".format(tel,event.index.obs_id,event.index.event_id)
disp = CameraDisplay(camgeom,title=title)
disp.image = event.simulation.tel[1].true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)

In [ ]:
# Creating image only with muon ring
edit_image = event.simulation.tel[1].true_image.copy()
true_image = event.simulation.tel[1].true_image
print(np.count_nonzero(edit_image))
# INDICES for central contamination of event 93 from obs94 protonMST file
indices1 = [i for i in range(160,175)]
indices2 = [i for i in range(1243, 1260)]
indices3 = [i for i in range(180,192)]
indices4 = [i for i in range(196,200)]
indices = indices1+indices2+indices3+indices4
#np.put(edit_image, indices1, [0])
#np.put(edit_image, indices2, [0])
#np.put(edit_image, indices3, [0])
np.put(edit_image, indices, [0])
print(np.count_nonzero(edit_image))

In [ ]:
# Creating the image of contamination, all pixels which are not the signal make equal to zero
cont_image =  event.simulation.tel[1].true_image.copy()
print(cont_image)
print(np.count_nonzero(cont_image))
ind = [i for i in range(1764)]
cont_ind = list(set(ind)-set(indices))
np.put(cont_image, cont_ind, [0]) 
print(cont_image)
print(np.count_nonzero(cont_image))

In [ ]:
plt.figure(figsize = (9,6))
camgeom = source.subarray.tel[1].camera.geometry
#title="CT{}, run {} event {}".format(tel,event.index.obs_id,event.index.event_id)
disp = CameraDisplay(camgeom,title=title)
#disp.image = edit_image
disp.image = cont_image
#disp.image = true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=False, gridspec_kw={'width_ratios': [1, 1]}, figsize=(12,5))
fig.subplots_adjust(hspace=0.2)
ax = axes[0]
for pix_id in range(0,1760):
    if pix_id in indices:
        continue
    else:
        ax.plot(event.r0.tel[1].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
ax.set_ylabel("ADC counts")
ax.set_xlabel("Samples")
ax.set_title("Only Ring")
#ax.set_ylim(320,1100)
bx = axes[1]
for pix_id in indices:
    plt.plot(event.r0.tel[1].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
#bx.set_ylabel("ADC counts")
#bx.set_ylim(320,1100)
bx.set_xlabel("Samples")
bx.set_title("Contamination")
plt.show()
#plt.savefig("/Users/vdk/WaveformsRingContaminationWithBackground")

In [ ]:
contaminated_waveform = np.array([])
test_w = []
for pix_id in indices:
    #np.append(contaminated_waveform, event.r0.tel[1].waveform[0][pix_id], axis = 0)
    test_w.append(event.r0.tel[1].waveform[0][pix_id])

#contaminated_waveform
#indices
#event.r0.tel[1].waveform[0][165]
np.argmax(test_w, axis = 0)
#test_w[2]

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=False, gridspec_kw={'width_ratios': [1, 1]}, figsize=(12,5))
fig.subplots_adjust(hspace=0.2)
dx = axes[0]
camgeom = source.subarray.tel[1].camera.geometry
disp = CameraDisplay(camgeom,title=title, ax = dx)
disp.image = edit_image
#disp.image = true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
dx.set_title("Extracted muon ring")
cx = axes[1]
camgeom = source.subarray.tel[1].camera.geometry
disp = CameraDisplay(camgeom,title=title, ax = cx)
disp.image = cont_image
#disp.image = true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
cx.set_title("Extracted contamination")
# Array for broken pixels
bool_cont_image = np.array(cont_image, dtype=bool)
bool_ring_image = np.array(edit_image, dtype=bool)
#plt.show()
#plt.savefig("/Users/vdk/DistinctRingContamination")

In [ ]:
from ctapipe.image.extractor import ImageExtractor

calib = CameraCalibrator(image_extractor_type="NeighborPeakWindowSum",subarray = source.subarray)
calib(event)

image_extractor = ImageExtractor.from_name("NeighborPeakWindowSum", subarray=source.subarray)
#image_extractor = ImageExtractor.from_name("GlobalPeakWindowSum", subarray=source.subarray)
image_extractor.local_weight = 0
image_extractor.window_shift = 3
image_extractor.window_width = image_extractor.window_shift.tel[1]*2+1
image_extractor

In [ ]:
broken_pixels = np.array([False for i in event.dl0.tel[1].waveform])
dl1_contamination = image_extractor(event.dl0.tel[1].waveform, tel_id = 1, selected_gain_channel=0, broken_pixels = bool_ring_image)
dl1_ring = image_extractor(event.dl0.tel[1].waveform, tel_id = 1, selected_gain_channel=0, broken_pixels = bool_cont_image)
#dl1 = image_extractor(cont_image, tel_id = 1, selected_gain_channel=0, broken_pixels = broken_pixels)


In [ ]:
plt.figure(figsize = (9,6))
camgeom = source.subarray.tel[1].camera.geometry
title="Extracted Image"
disp = CameraDisplay(camgeom,title=title)
disp.image = dl1_contamination.image
#disp.image = true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)

In [ ]:
plt.figure(figsize = (9,6))
camgeom = source.subarray.tel[1].camera.geometry
title="Extracted Image"
disp = CameraDisplay(camgeom,title=title)
disp.image = dl1_ring.image
#disp.image = true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)

In [ ]:
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="NeighborPeakWindowSum",subarray = source.subarray)
event.dl1.tel[1] = dl1_ring
#calib(event)
image_processor(event)
muon_processor(event)

In [ ]:
print(event.muon.tel[1])

In [ ]:
pixel_range = [i for i in range(len(event.dl1.tel[1].image))]
#dl1_contamination.peak_time

plt.scatter(pixel_range, dl1_ring.image, alpha = 0.5)
plt.scatter(pixel_range, dl1_contamination.image, alpha = 0.5)


In [ ]:
for pix_id in range(0,1764):
    plt.plot(dl1_ring.waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')

In [ ]:
print(min(dl1_contamination.peak_time))
print(max(dl1_contamination.peak_time))
print(min(dl1_ring.peak_time))
print(max(dl1_ring.peak_time))

print(np.argmax(max(dl1_contamination.peak_time)))
print(np.argmax(max(dl1_ring.peak_time)))

In [ ]:
peak_ring = dl1_ring.peak_time * source.subarray.tel[1].camera.readout.sampling_rate.to_value(u.GHz)
peak_cont = dl1_contamination.peak_time * source.subarray.tel[1].camera.readout.sampling_rate.to_value(u.GHz)
np.around(peak_ring)
pixel_range = [i for i in range(len(event.dl1.tel[1].image))]
#dl1_contamination.peak_time

#plt.scatter(np.around(peak),pixel_range)
plt.rcParams.update({'font.size': 12, 'figure.figsize': (6,6)})
histplot(np.around(peak_cont), stat='count', bins = 10)
#plt.xscale('log')
#plt.yscale('log')
plt.title('Histogram for the peak sample')
plt.xlabel('Sample')
plt.grid()
#plt.savefig(f"/Users/vdk/EnergyHistogram_obs101.png")

In [ ]:
pixel_range = [i for i in range(len(event.dl1.tel[1].image))]
#dl1_contamination.peak_time

plt.scatter(pixel_range, dl1_contamination.peak_time)

In [ ]:
plt.scatter(pixel_range, dl1_ring.peak_time)

In [ ]:
max_indices = np.where(dl1_contamination.peak_time == dl1_ring.peak_time.max())
len(max_indices[0])

In [ ]:
plt.figure(figsize = (9,6))
camgeom = source.subarray.tel[1].camera.geometry
title="Intensity cut < mean+10% value"
disp = CameraDisplay(camgeom,title=title)
disp.image = event.dl1.tel[1].image
#disp.image = true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)

In [ ]:
dl1 = DL1CameraContainer(
                image=waveforms[..., 0].astype(np.float32),
                peak_time=np.zeros(n_pixels, dtype=np.float32),
                is_valid=True,
            )
extractor = self.image_extractors[self.image_extractor_type.tel[tel_id]]
dl1 = extractor(
                waveforms,
                tel_id=tel_id,
                selected_gain_channel=selected_gain_channel,
                broken_pixels=broken_pixels,
            )



In [ ]:
event.r0.tel[1].waveform[0]

In [ ]:
cont_image

In [ ]:
edit_image

In [ ]:
bool_cont_image = np.array(cont_image, dtype=bool)
bool_ring_image = np.array(edit_image, dtype=bool)

In [ ]:
bool_arr

In [ ]:
filename = '/Users/vdk/MST_work/data_muons/mono_proton_simtel/proton_20deg_0deg_run2___cta-prod6-paranal-2147m-Paranal-nshow-3000-0-LSTs-1-MSTs-0-SSTs-NSCAT_5_CSCAT_300_EMIN_1_EMAX_200_after_muontag.simtel.zst'
#filename = '/Users/vdk/MST_work/data_muons/mono_muon_simtel/muon_20deg_0deg_run0___cta-prod6-paranal-2147m-Paranal-0-LSTs-1-MSTs-0-SSTs-NSCAT_1_CSCAT_8_EMIN_0.005_EMAX_1_FIXCHI_600_VIEWCONE_3_after_muontag.simtel.zst'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)



for i,j in enumerate(event_iterator):
    if i == 93:
        event = j
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(
                image_extractor_type="NeighborPeakWindowSum",
                subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        print(event.muon.tel[1].efficiency)
        fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
        fig.subplots_adjust(hspace=0.2)
        ax = axes[0]
        camgeom = source.subarray.tel[1].camera.geometry
        title="TrueImage run {} if {} fit {} width {}".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = ax)
        disp.image = j.simulation.tel[1].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        bx = axes[1]
        for pix_id in range(0,1764):
            bx.plot(j.r0.tel[1].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
        bx.set_ylabel("ADC counts")
        bx.set_xlabel("Samples")
        bx.set_title("Waveforms")
        plt.show()
        #plt.savefig(f"/Users/vdk/MST_ProtonImages_NewQualityCheck/TrueImage+Waveform_mstPROTON_event{j.index.event_id}_obs{j.index.obs_id}_numer{i}")
        #plt.savefig(f"/Users/vdk/TrueImage+Waveform_mstMUON_event{j.index.event_id}_obs{j.index.obs_id}")
        break
        #if i == 150:
        #    break

plt.close()